1. CSR matrix가 정상적으로 만들어졌다.  
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.  
  
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
  
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.  

# 영화 추천 실습
  
유저 평점 데이터가 크기별로 존재, MovieLens 1M Dataset 사용을 권장  
별점은 explicit 데이터지만 implicit 데이터로 간주하고 테스트할 수도 있다.(그래서??)  
별점은 시청횟수로 해석하며, 3점 미만은 선호하지 않는다고 가정  
  
  
1.데이터 준비와 전처리  
2.분석: ratings에 있는 유니크한 영화 개수와 사용자 수, 가장 인기 있는 영화 30개(인기순)  
3.선호영화 5개를 골라서 추가  
4.csr matrix 작성  
5.als_model=AlternatingLeastSquares모델을 구성 및 훈련  
6.선호영화 5개와 그 외의 영화 하나를 골라 모델이 예측한 나의 선호도 파악  
7.영화 추천받기(내가 좋아하는 영화와 비슷한 영화)  
8.영화 추천받기(내가 좋아할 만한 영화)  

### 데이터 준비와 전처리

In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd

In [2]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
ratings=ratings[['user_id', 'movie_id','ratings']]
ratings.head(10)

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [4]:
# 3점 이상만 남기기
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)
ratings[ratings['ratings']<=2]

,user_id,movie_id,ratings


In [5]:
# 원본 데이터 대비 남은 데이터 비율(=1, 2점을 준 후기 비율)
print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


약 20%만이 1점과 2점을 주었음을 알 수 있다.  

In [6]:
# rating->counts 변환
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [7]:
ratings

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies['movie_id'].nunique()

3883

### 분석

In [10]:
# 회원들이 관람한 영화의 수, 전체 영화의 수보다 약간 작다.
ratings['movie_id'].nunique('counts')

3628

In [11]:
# 영화를 관람한 회원의 총 수
ratings['user_id'].nunique()

6039

In [12]:
# 인기 영화 30선
pop_mov=ratings.groupby('movie_id')['user_id'].count().sort_values(ascending=False).copy()

pd.merge(pop_mov.head(30), movies, how='left', on='movie_id')

,movie_id,user_id,title,genre
0,2858,3211,American Beauty (1999),Comedy|Drama
1,260,2910,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1196,2885,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
3,1210,2716,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
4,2028,2561,Saving Private Ryan (1998),Action|Drama|War
5,589,2509,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
6,593,2498,"Silence of the Lambs, The (1991)",Drama|Thriller
7,1198,2473,Raiders of the Lost Ark (1981),Action|Adventure
8,1270,2460,Back to the Future (1985),Comedy|Sci-Fi
9,2571,2434,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [13]:
# 선호 영화 row 추가 (5개)
my_favorite = [1580 , 2571 ,480 ,318 ,1] # 맨인블렉, 메트릭스, 쥬라기공원, 쇼생크탈출, 토이스토리
my_playlist = pd.DataFrame({'user_id': [9999]*5, 'movie_id': my_favorite, 'counts':[4, 5, 5, 5,4]})
if not ratings.isin({'user_id':[9999]})['user_id'].any():  
    ratings = ratings.append(my_playlist)                   

ratings.tail(10)       

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,9999,1580,4
1,9999,2571,5
2,9999,480,5
3,9999,318,5
4,9999,1,4


### CSR Matrix 작성 및 훈련

In [14]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = max(ratings['user_id'].unique())
num_movie = max(ratings['movie_id'].unique())

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user+1, num_movie+1))
csr_data

<10000x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [15]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 선호도 파악

In [19]:
my_id9999, my_movie1 = 9999, 1
id_9999_vector, movie_1_vector = als_model.user_factors[my_id9999], als_model.item_factors[my_movie1]

In [20]:
id_9999_vector.shape

(100,)

In [21]:
movie_1_vector.shape

(100,)

In [22]:
np.dot(id_9999_vector, movie_1_vector)

0.4390508

선호영화 5개와 그 외의 영화 하나를 골라 모델이 예측한 나의 선호도 파악

In [23]:
# my_favorite영화와 (맨인블렉, 메트릭스, 쥬라기공원, 쇼생크탈출, 토이스토리) [1580 , 2571 ,480 ,318 ,1]
# 그 외의 영화(593, 양들의 침묵)의 선호도 출력
import copy

test_movie_list=copy.deepcopy(my_favorite)
test_movie_list.append(593)

In [24]:
# 선호도 출력.. 생각보다 선호도가 높게 나오지도 않았고 선호하지 않는 영화로 둔 양들의 침묵과 차이도 크지는 않았다.
for i in test_movie_list:
    movie_2_vector = als_model.item_factors[i]
    print(movies.loc[movies['movie_id']==i, ['title']], end=': ')
    print(np.dot(id_9999_vector, movie_2_vector))

                    title
1539  Men in Black (1997): 0.6519561
                   title
2502  Matrix, The (1999): 0.5807049
                    title
476  Jurassic Park (1993): 0.6929102
                                title
315  Shawshank Redemption, The (1994): 0.44176355
              title
0  Toy Story (1995): 0.4390508
                                title
589  Silence of the Lambs, The (1991): 0.41019475


In [36]:
# 내가 좋아하는 영화와 비슷한 영화 추천받기

whole_movie=[]

for i in my_favorite:
    similar=als_model.similar_items(i, N=5)
    
    for j in similar:
        if j[0] in my_favorite or j[0] in whole_movie: # my_favorite와 이미 출력한 영화는 무시
            continue
        print(movies.loc[movies['movie_id']==j[0], ['title']], end=': ')
        print(j[1])
        whole_movie.append(j[0])


                                 title
585  Terminator 2: Judgment Day (1991): 0.61625963
                    title
2847  Total Recall (1990): 0.5997015
                             title
770  Independence Day (ID4) (1996): 0.51668996
                    title
453  Fugitive, The (1993): 0.5622823
                       title
1220  Terminator, The (1984): 0.56075644
                                title
589  Silence of the Lambs, The (1991): 0.8011237
                       title
523  Schindler's List (1993): 0.75114095
                   title
293  Pulp Fiction (1994): 0.71922714
                         title
1656  Good Will Hunting (1997): 0.68219507
                   title
3045  Toy Story 2 (1999): 0.7949175
                     title
2286  Bug's Life, A (1998): 0.6002237
              title
584  Aladdin (1992): 0.5702385
          title
33  Babe (1995): 0.5694522


In [55]:
# 내가 좋아할 만한 영화 추천받기

user = 999
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

for i in artist_recommended:
    print(movies.loc[movies['movie_id']==i[0], ['title']], end=': ')
    print(i[1])

                  title
430  Cliffhanger (1993): 1.1714292
             title
821  Ransom (1996): 1.1149445
                       title
2199  Few Good Men, A (1992): 1.0845467
                         title
490  Executive Decision (1996): 0.9892531
                title
20  Get Shorty (1995): 0.9725121
                    title
2951  Falling Down (1993): 0.9583216
                   title
1239  Stand by Me (1986): 0.94216275
                  title
623  Primal Fear (1996): 0.9410872
                                  title
525  Searching for Bobby Fischer (1993): 0.9226077
                                            title
2046  Indiana Jones and the Temple of Doom (1984): 0.91541266
                title
3458  Predator (1987): 0.9058231
               title
2421  Payback (1999): 0.8970287
                  title
93  Broken Arrow (1996): 0.88859415
                       title
1574  Peacemaker, The (1997): 0.88618964
             title
3181  Alive (1993): 0.8760141
             title
77

어느 정도는 괜찮았지만 내가 입력한 것들과 거리가 좀 있다는 생각도 든다. 반만 맞은 느낌이다.

회고  
지난 과제에서 우려했던 것과 다르게, 과제의 난이도는 크게 어렵다는 느낌은 아니었다. layer를 쌓거나 그런 방식의 학습은 아니었지만 직접 데이터를 만지고 조작이 가능한 정도의 난이도였기 때문에 오히려 만족스러웠다.  
다만, 결과는 여전히 별로라 개선을 할 수 있었으면 좋았을 텐데 그렇게까지는 잘 되지 않았다. 뭔가 잘 안된건 아닌데 '아, 이거 괜찮은데?' 싶다가도 하나씩 이상한 값이 들어가 있거나, 과제 제출에는 나와 있지 않지만 토이 스토리에 대해 쥬만지(둘 다 가정용 영화)에 대한 점수가 1%이하로 매우 낮은 경우가 발생하거나 내가 지정한 선호 영화와는 전혀 관계 없는 영화들이 선정되는 경우도 가끔 발견되어서 꿉꿉한 부분이 있다. 반복하여 학습하다보면 점수가 꽤 다르게 나오는 경우가 있어서 당황스러웠다.  
그러나 데이터를 이것저것 만져본 것으로 만족하고 제출하겠다! 방학이니까!  